In [16]:
!pip install pandas
!pip install google.colab

You should consider upgrading via the 'C:\Users\evalv\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\evalv\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~ornado\\speedups.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Users\evalv\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.



  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
Using legacy 'setup.py install' for google.colab, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pandas, since package 'wheel' is not installed.
Using legacy 'setup.py install' for portpicker, since package 'wheel' is not installed.
Using legacy 'setup.py install' for simplegeneric, since package 'wheel' is not installed.
Using legacy 'setup.py install' for tornado, since package 'wheel' is not installed.
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: tornado
    Found existing installation: tornado 6.3.2
    Uninstalling tornado-6.3.2:
      Successfully uninstalled tornado-6.3.2
    Running setup.py install for tornado: started
    Running setup.py install for tornado: finished with status 'done'


In [17]:
import pandas as pd

In [18]:
test_comments_post1 = ['1comment1', '1comment2']
test_comments_post2 = ['2comment1', '2comment2']
comments_post1_df = pd.DataFrame({'comment': test_comments_post1})
comments_post2_df = pd.DataFrame({'comment': test_comments_post2})
path_main = 'data'
comments_post1_df.to_csv(path_main + '\\comments1.csv')
comments_post2_df.to_csv(path_main + '\\comments2.csv')


comments_post1_df

,comment
0,1comment1
1,1comment2


In [19]:
comments_post2_df

,comment
0,2comment1
1,2comment2


In [19]:
from google.colab import drive
drive.mount('data\comments1.csv')
drive.mount('data\comments2.csv')